In [1]:
MODEL = "qwen3:1.7b"
# MODEL = "gpt-oss-bn-json"
PROMPT = """Consider this question: '{question}'. What are the two nodes in this question? Make sure to correctly output the names of nodes exactly as mentioned in the network and in the order as the question mentioned. For example, if the question mentioned "A and B" then the two nodes are fromNode: A, toNode: B; or if the question mentioned "Smoking and Cancer" then the two nodes are fromNode: Smoking, toNode: Cancer. Answer in JSON format."""

# User Query Input

In [2]:
import requests
import json
from IPython.display import display, Markdown, clear_output

def answer_this_prompt(prompt, stream=False, model=MODEL, temperature=0, format=None):
    payload = {
        "prompt": prompt,
        "model": model,
        "temperature": temperature,
        "max_new_tokens": 50, # only when stream = False work
        "format": format
    }
    headers = {
        'Content-Type': 'application/json'
    }
    endpoint = "http://localhost:11434/api/generate"

    # Send the POST request with streaming enabled
    with requests.post(endpoint, headers=headers, json=payload, stream=True) as response:
        if response.status_code == 200:
            try:
                # Process the response incrementally
                full_response = ""
                for line in response.iter_lines(decode_unicode=True):
                    if line.strip():  # Skip empty lines
                        response_json = json.loads(line)
                        chunk = response_json.get("response", "")
                        full_response += chunk
                        
                        # Render the response as Markdown
                        if stream:
                            clear_output(wait=True)
                            display(Markdown(full_response))
                        
                return full_response
            except json.JSONDecodeError as e:
                return "Failed to parse JSON: " + str(e)
        else:
            return "Failed to retrieve response: " + str(response.status_code)
        
def multiple_answer_this_prompt(prompt, stream=False, model=MODEL, temperature=0, format=None, n_answers=1):
    answers = []
    for _ in range(n_answers):
        answer = answer_this_prompt(prompt, stream=stream, model=model, temperature=temperature, format=format)
        answers.append(answer)
    return answers

In [5]:
# Example usage
# ans = answer_this_prompt("What is the Big Bang theory?", stream=True)
# print("------------------------")
# # print(ans)

In [3]:
bn_path = "./nets/collection/"
from bni_netica.bni_netica import *
from bni_netica.bni_netica import Net

CancerNeapolitanNet = Net(bn_path+"Cancer Neapolitan.neta")
ChestClinicNet = Net(bn_path+"ChestClinic.neta")
ClassifierNet = Net(bn_path+"Classifier.neta")
CoronaryRiskNet = Net(bn_path+"Coronary Risk.neta")
FireNet = Net(bn_path+"Fire.neta")
MendelGeneticsNet = Net(bn_path+"Mendel Genetics.neta")
RatsNet = Net(bn_path+"Rats.neta")
WetGrassNet = Net(bn_path+"Wet Grass.neta")
RatsNoisyOr = Net(bn_path+"Rats_NoisyOr.dne")
Derm = Net(bn_path+"Derm 7.9 A.dne")

Loading Netica


In [9]:
BN = ""

for node in ChestClinicNet.nodes():
	# print(f"{node.name()} -> {[child.name() for child in node.children()]}")
	BN += f"{node.name()} -> {[child.name() for child in node.children()]}\n"
	
print(BN)

VisitAsia -> ['Tuberculosis']
Tuberculosis -> ['TbOrCa']
Smoking -> ['Cancer', 'Bronchitis']
Cancer -> ['TbOrCa']
TbOrCa -> ['XRay', 'Dyspnea']
XRay -> []
Bronchitis -> ['Dyspnea']
Dyspnea -> []



In [4]:
from pydantic import BaseModel

class Query(BaseModel):
  fromNode: str
  toNode: str

In [7]:
# fromNode = "Smoking"
# toNode = "Cancer"
# input = PROMPT.format(BN=BN, fromNode=fromNode, toNode=toNode) 

# ans = answer_this_prompt(input, format=Query.model_json_schema())
# # ans = answer_this_prompt(input)
# # print(ans)
# query = Query.model_validate_json(ans)
# print(query)

In [ ]:
# queryFromNode = query.fromNode
# queryToNode = query.toNode
# print(f"Is {queryFromNode} connected to {queryToNode}?")

Is Smoking connected to Cancer?


In [5]:
def isConnected(net, fromNode, toNode):
  relatedNodes = net.node(fromNode).getRelated("d_connected")
  for node in relatedNodes:
    if node.name() == toNode:
      return True
  return False

In [ ]:
# print(f"Is {queryFromNode} connected to {queryToNode}? {'Yes' if isConnected(ChestClinicNet, queryFromNode, queryToNode) else 'No'}")

Is Smoking connected to Cancer? Yes


In [6]:
def draftPrompt(PROMPT, question, net, fromNode, toNode):
    BN = ""
    for node in net.nodes():
        BN += f"{node.name()} -> {[child.name() for child in node.children()]}\n"

    question = question.format(BN=BN, fromNode=fromNode, toNode=toNode)
    # print('Question:', question)

    return PROMPT.format(question=question)

In [7]:
def explainConnection(PROMPT, question, net, fromNode, toNode):
    PROMPT = draftPrompt(PROMPT, question, net, fromNode, toNode)
    # print('PROMPT:', PROMPT)
    # print('------------------------------------------------')
    preTemplate = "This is the answer template: "
    postTemplate = " Answer the given question exactly as the given template."
    template = ""
    if isConnected(net, fromNode, toNode):
        template = f"Yes, {fromNode} is d-connected to {toNode}, which means that entering evidence for {fromNode} would change the probability of {toNode} and vice versa."
    else:
        template = f"No, {fromNode} is not d-connected to {toNode}, which means that entering evidence for {fromNode} would not change the probability of {toNode}."

    return answer_this_prompt(PROMPT + preTemplate + template + postTemplate)

In [ ]:
# print(explainConnection(ChestClinicNet, queryNode1, queryNode2))

Yes, Smoking is d-connected to Cancer, which means that entering evidence for Smoking would change the probability of Cancer and vice versa.


# CHECKING CHECKING CHECKING

In [9]:
def correctIdentification(PROMPT, question, net, fromNode, toNode):
    input = draftPrompt(PROMPT, question, net, fromNode, toNode)
    ans = answer_this_prompt(input, format=Query.model_json_schema())
    query = Query.model_validate_json(ans)
    queryFromNode = query.fromNode
    queryToNode = query.toNode

    sameIden = queryFromNode == fromNode and queryToNode == toNode
    reverseIden = queryFromNode == toNode and queryToNode == fromNode
    correct = sameIden or reverseIden

    return correct, queryFromNode, queryToNode

In [ ]:
# node1 = "Smoking"
# node2 = "Cancer"
# print(f"Correct identification: {correctIdentification(ChestClinicNet, node1, node2)}")

Correct identification: True


In [10]:
import random 
def pickTwoRandomNodes(net):
    nodes = net.nodes()
    if len(nodes) < 2:
        return None, None
    node1, node2 = random.sample(nodes, 2)
    return node1.name(), node2.name()

In [ ]:
# print(pickTwoRandomNodes(ChestClinicNet))

('Dyspnea', 'XRay')


In [11]:
def printNet(net):
    for node in net.nodes():
        print(f"{node.name()} -> {[child.name() for child in node.children()]}")

In [ ]:
listOfNets = [CancerNeapolitanNet, ChestClinicNet, ClassifierNet, CoronaryRiskNet, FireNet, MendelGeneticsNet, RatsNet, WetGrassNet, RatsNoisyOr, Derm]
total = 0
correct = 0
question = """In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}?"""

for net in listOfNets:
    for _ in range(10):
      total += 1
      fromNode, toNode = pickTwoRandomNodes(net)
      if fromNode and toNode:
          correctIdentified, queryFromNode, queryToNode = correctIdentification(PROMPT, question, net, fromNode, toNode)
          if correctIdentified:
            correct += 1
          else:
            print(f"Incorrect identification for {net.name()}")
            printNet(net)
            print()
            print("Expected:", fromNode, "->", toNode)
            print("Reality:", queryFromNode, "->", queryToNode)
            print("----------------------------------------------------")

print(f"Total: {total}, Correct: {correct}, Accuracy: {correct/total:.2%}")

Incorrect identification for ChestClinic
VisitAsia -> ['Tuberculosis']
Tuberculosis -> ['TbOrCa']
Smoking -> ['Cancer', 'Bronchitis']
Cancer -> ['TbOrCa']
TbOrCa -> ['XRay', 'Dyspnea']
XRay -> []
Bronchitis -> ['Dyspnea']
Dyspnea -> []

Expected: VisitAsia -> Bronchitis
Reality: VisitAsia -> Bronchopsis
----------------------------------------------------
Incorrect identification for Coronary_Artery_Disease_Risk
Sex -> ['Smoking_status', 'LVH', 'Coronary_artery_disease']
Smoking_status -> ['Coronary_artery_disease']
Systolic_Blood_Pressure -> ['LVH', 'Coronary_artery_disease']
Age -> ['LVH', 'Coronary_artery_disease']
LVH -> ['Coronary_artery_disease']
Diabetes_mellitus -> ['Coronary_artery_disease']
Total_chol -> ['Coronary_artery_disease']
HDL_Status -> ['Coronary_artery_disease']
Coronary_artery_disease -> []

Expected: Smoking_status -> Sex
Reality: Smoking -> i2
----------------------------------------------------
Incorrect identification for Wet_Grass
Rain -> ['Sprinkler', 'Grass

In [17]:
question_test = """In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}?"""
PROMPT_TEST = """Consider this question: '{question}'. What are the two nodes in this question? Make sure to correctly output the names of nodes. Answer in JSON format."""
net_test = CancerNeapolitanNet
fromNode_test, toNode_test = pickTwoRandomNodes(net_test)
correctIdentified, queryFromNode, queryToNode = correctIdentification(PROMPT_TEST, question_test, net_test, fromNode_test, toNode_test)

print(correctIdentified)
print(queryFromNode)
print(queryToNode)

Question: In this Bayesian Networks: A -> ['B', 'C']
B -> ['D']
C -> ['D', 'E']
D -> []
E -> []
, is D connected to B?
True
D
B


In [12]:
questions = [
    """In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}?""",
    "Within the Bayesian Network {BN}, does a path exist from {fromNode} to {toNode}?",
    "In the graph {BN}, can information flow from {fromNode} to {toNode}?", # top perform 
    "Are {fromNode} and {toNode} dependent in the Bayesian Network {BN}?",
    "In {BN}, is there any direct or indirect connection between {fromNode} and {toNode}?",
    "Can {fromNode} influence {toNode} in the Bayesian Network {BN}?",
    "Is {toNode} reachable from {fromNode} in the structure of {BN}?",
    "Does {BN} contain a path that links {fromNode} to {toNode}?",
    "Are there any edges—direct or through other nodes—connecting {fromNode} and {toNode} in {BN}?",
    "Is {toNode} conditionally dependent on {fromNode} in the Bayesian Network {BN}?",
    "Within {BN}, is {fromNode} an ancestor of {toNode}?"
]

In [14]:
listOfNets = [CancerNeapolitanNet, ChestClinicNet, ClassifierNet, CoronaryRiskNet, FireNet, MendelGeneticsNet, RatsNet, WetGrassNet, RatsNoisyOr, Derm]

accuracy = []
responses = []

for question in questions:
  total = 0
  correct = 0
  print(f"Question: {question}")
  for net in listOfNets:
      for _ in range(5):
        total += 1
        fromNode, toNode = pickTwoRandomNodes(net)
        if fromNode and toNode:
            correctIdentified, queryFromNode, queryToNode = correctIdentification(PROMPT, question, net, fromNode, toNode)
            # response = explainConnection(question, net, fromNode, toNode)
            # responses.append(response)
            # print(response)
            
            if correctIdentified:
              correct += 1
            else:
              print(f"Incorrect identification for {net.name()}")
              printNet(net)
              print()
              print("Expected:", fromNode, "->", toNode)
              print("Reality:", queryFromNode, "->", queryToNode)
              print("----------------------------------------------------")

  print(f"Total: {total}, Correct: {correct}, Accuracy: {correct/total:.2%}")
  accuracy.append(correct/total)
  print("<------------------------------------------------------------------------->")

print('===> Average accuracy:', sum(accuracy)/len(accuracy))

Question: In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}?
Incorrect identification for Cancer_Neapolitan
A -> ['B', 'C']
B -> ['D']
C -> ['D', 'E']
D -> []
E -> []

Expected: D -> B
Reality: A -> B
----------------------------------------------------
Incorrect identification for Cancer_Neapolitan
A -> ['B', 'C']
B -> ['D']
C -> ['D', 'E']
D -> []
E -> []

Expected: E -> B
Reality: A -> E
----------------------------------------------------
Incorrect identification for Classifier
Class -> ['Obsv', 'Result']
Obsv -> ['Classifier']
Classifier -> ['Result']
Result -> []

Expected: Class -> Obsv
Reality: Obsv -> Classifier
----------------------------------------------------
Incorrect identification for Rats
Ecstazine -> ['Neurofill', 'Social_Activity']
Neurofill -> ['Social_Activity']
Social_Activity -> ['Squeaking']
Squeaking -> []

Expected: Neurofill -> Social_Activity
Reality: Ecstazine -> Social_Activity
----------------------------------------------------
Incorr

In [ ]:
git pull --rebase

In [19]:
listOfNets = [CancerNeapolitanNet, ChestClinicNet, ClassifierNet, CoronaryRiskNet, FireNet, MendelGeneticsNet, RatsNet, WetGrassNet, RatsNoisyOr, Derm]

accuracy = []
responses = []

for question in questions:
  total = 0
  correct = 0
  # print(f"Question: {question}")
  for net in listOfNets:
      for _ in range(1):
        total += 1
        fromNode, toNode = pickTwoRandomNodes(net)
        if fromNode and toNode:
            correctIdentified, queryFromNode, queryToNode = correctIdentification(question, net, fromNode, toNode)
            response = explainConnection(question, net, fromNode, toNode)
            responses.append(response)
            print('Response:',response)
            print('------------------------------------------------')
            print()
            
            if correctIdentified:
              correct += 1
            else:
              print(f"Incorrect identification for {net.name()}")
              printNet(net)
              print()
              print("Expected:", fromNode, "->", toNode)
              print("Reality:", queryFromNode, "->", queryToNode)
              print("----------------------------------------------------")

  print(f"Total: {total}, Correct: {correct}, Accuracy: {correct/total:.2%}")
  accuracy.append(correct/total)
  print("<------------------------------------------------------------------------->")

print('===> Average accuracy:', sum(accuracy)/len(accuracy))

Question: Consider this question: 'In this Bayesian Networks: A -> ['B', 'C']
B -> ['D']
C -> ['D', 'E']
D -> []
E -> []
, is information flow from A to C?'. What are the two nodes in this question? Make sure to correctly output the names of nodes. Answer in JSON format.
------------------------------------------------
Response: The user asks: "Consider this question: 'In this Bayesian Networks: A -> ['B', 'C'] B -> ['D'] C -> ['D', 'E'] D -> [] E -> [] , is information flow from A to C?'. What are the two nodes in this question? Make sure to correctly output the names of nodes. Answer in JSON format.This is the answer template: Yes, A is d-connected to C, which means that entering evidence for A would change the probability of C and vice versa. Answer the given question exactly as the given template."

So the user asks: "What are the two nodes in this question?" So they want two nodes: A and C. They want the answer exactly as the given template, but also they want to output the names 

KeyboardInterrupt: 